In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import scipy
import sobol_seq

from astropy.timeseries import LombScargle
import sys
sys.path.append('../sloscillations/TuRBO/')
from turbo import Turbo1, TurboM
from turbo.utils import from_unit_cube, latin_hypercube, to_unit_cube
import frequencies, mixed_modes_utils
from tqdm import tqdm_notebook as tqdm
from joblib import Parallel, delayed
import pandas as pd
pd.options.mode.chained_assignment = None

## Define save folders

In [ ]:
image_folder = '../results/images/'
samples_folder = '../results/samples/'
output_file = '../results/rotation_Mosser2016_ABBA.txt'
abba_psd_folder = '../data/KeplerRGpeakbagging/Kallinger_Vrard_BGCorr/'
abba_folder = '../data/KeplerRGpeakbagging/ModeFiles/'

## Define input datafile

In [ ]:
summary_df = pd.read_csv('../data/KeplerRGpeakbagging/summary.dat', delimiter=';').rename(columns={'ID': 'KIC'})
mos_q_df = pd.read_csv('Mosser_2016.dat', delim_whitespace=True)
mos_q_df = mos_q_df.rename(columns={'DPi1': 'dpi'})
summary_df = summary_df.merge(mos_q_df, how='inner', on='KIC')
summary_df = summary_df[summary_df.evo == 0] # For now pick only RGB

## here summary_df is the intersection of RGB stars peakbagged by Kallinger that are in the Mosser 2016 paper

# Preamble code for processing ABBA data:

In [ ]:
#' It is assumed that DeltaNu is in μHz
def DeltaPi1_from_DeltaNu_RGB(DeltaNu):
    # Compute Period spacing (in s) from deltanu
    return 60 + 1.7*DeltaNu

def Lor_model(pds, peak):
    return peak.height / (1 + ((pds.frequency.values - peak.freq)/ (peak.linewidth/2))**2) # add /2 factor to peak.linewidth

def sinc2_model(pds, peak):
    deltanu = np.mean(np.diff(pds.frequency.values))
    return peak.height * np.sinc((pds.frequency.values - peak.freq)/deltanu)**2

def fit_model(pds, peaks):

    model = np.ones_like(pds.frequency.values) # times 2?

    for i in range(len(peaks)):
        if np.isfinite(peaks.linewidth.iloc[i]):
            model += Lor_model(pds, peaks.iloc[i,])
        else:
            model += sinc2_model(pds, peaks.iloc[i, ])
    return model

def prepare_l1_peaks(peaks: pd.DataFrame,
                     evidence_cut: [float] = 0.0, height_cut: [float] = 0.0) -> pd.DataFrame:
    """
    Extract the mixed modes from the peaks dataframe.
    
    Parameters
    ----------
    peaks: pd.DataFrame
        Dataframe containing the detected peaks and parameters.
        
    summary: pd.DataFrame
        Dataframe containing the global stellar information.
    
    AIC_cut: Optional[float] = 0.0
        Cut to make in the Akaike Information Criterion if desired.
        
    height_cut: Optional[float] = 0.0
        Cut to make in the mode height if desired.
        
    Outputs
    -------
    pd.DataFrame
        Dataframe containing the mixed mode peaks and associated mode parameters.
    """
    peaks['x'] = ((peaks['freq'] % dnu - eps) / dnu) % 1
    # Don't want to include any modes near l=0 or 2s, this is why this and the step in the next cell is performed.
    x_range = [(np.minimum(np.min(peaks.loc[peaks['l'] == 0, 'x']), np.min(peaks.loc[peaks['l'] == 2, 'x'])) - 0.05) % 1,
               (np.maximum(np.max(peaks.loc[peaks['l'] == 0, 'x']), np.max(peaks.loc[peaks['l'] == 2, 'x'])) + 0.05) % 1]
    
    l1_peaks = peaks.loc[(peaks.l == 1) | ~np.isfinite(peaks.l), ]
    l1_peaks['x'] = ((l1_peaks['freq'] % dnu - eps) / dnu) % 1
    
    if x_range[0] < x_range[1]:
        l1_peaks = l1_peaks.loc[(l1_peaks['x'] < x_range[1]) | (l1_peaks['x'] > x_range[0]), ] # changed to OR for HeB
    else:
        l1_peaks = l1_peaks.loc[(l1_peaks['x'] > x_range[1]) & (l1_peaks['x'] < x_range[0]), ] # this is AND for RGB
#     print(l1_peaks)
    l1_peaks = l1_peaks.loc[(l1_peaks['height'] > height_cut), ]
    l1_peaks = l1_peaks.loc[(l1_peaks['ev1'] > evidence_cut), ]

    return l1_peaks

## Loss function

In [ ]:
from scipy.spatial.distance import cdist
from scipy.spatial import KDTree

def kdtree_match(X_model, X_real, DPi1, numax, delta_nu, weights):
    c1 = np.vstack((X_model[:,0]/DPi1, (X_model[:,1]-numax)/delta_nu)).T
    c2 = np.vstack((X_real[:,0]/DPi1, (X_real[:,1]-numax)/delta_nu)).T

    kdtree = KDTree(c1)
    distances, idx = kdtree.query(c2, k=1)
    dist = np.median(distances*weights)
    return dist

In [ ]:
class freq_tau_rot_triplet:
    def __init__(self, init_dpi, init_q, freqs_class, inp_freqs, weights):
        self.lb = np.array([init_dpi*0.98, max(0, init_q-0.1), -0.4, 0.1]) #or another judicious choice
        self.ub = np.array([init_dpi*1.02, init_q+0.1, 0.4, 1.0]) ## RGB
        
#         self.lb = np.array([init_dpi*0.98, 0.05, -0.5, 0.0]) 
#         self.ub = np.array([init_dpi*1.02, 0.6, 0.2, 0.1]) # HeB
        self.freqs = freqs_class
        self.inp_freqs = inp_freqs
        self.weights = weights
        
    def __call__(self, x):
        assert len(x) == 4
        assert x.ndim == 1
        assert np.all(x <= self.ub) and np.all(x >= self.lb)
        
        dpi1, coupling, eps_g, split = x[0], x[1], x[2], x[3]
        
        params = {'calc_l0': True, # copy of params with DPi1 set to candidate dpi from loop
            'calc_l2': True, 
            'calc_l3': False, 
            'calc_nom_l1': True, 
            'calc_mixed': True, 
            'calc_rot': False,
            'DPi1': dpi1,
            'coupling': coupling, 
            'eps_g': eps_g, 
            'split_core': split,
            'split_env': 0.0,
            'l': 1,
            }
    
        self.freqs(params)

        self.freqs.generate_tau_values()

        # Compute tau from the zeta value just computed
        real_tau = mixed_modes_utils.peaks_stretched_period(self.inp_freqs, 
                                                                 self.freqs.frequency, 
                                                                 self.freqs.tau)

        freqs_p1 = self.freqs.l1_mixed_freqs + self.freqs.l1_zeta * split
        freqs_n1 = self.freqs.l1_mixed_freqs - self.freqs.l1_zeta * split

        tau_p1 = mixed_modes_utils.peaks_stretched_period(freqs_p1, self.freqs.frequency, self.freqs.tau)
        tau_n1 = mixed_modes_utils.peaks_stretched_period(freqs_n1, self.freqs.frequency, self.freqs.tau)

        model_freqs = np.c_[self.freqs.l1_mixed_freqs, freqs_p1, freqs_n1]
        model_tau = np.c_[self.freqs.l1_mixed_tau, tau_p1, tau_n1]   

#         # #triplet case
        X = np.c_[np.r_[model_tau[:,0],
                        model_tau[:,1] - self.freqs.shift * self.freqs.DPi1, 
                        model_tau[:,2] - self.freqs.shift * self.freqs.DPi1], 
                  np.r_[model_freqs[:,0],
                        model_freqs[:,1], 
                        model_freqs[:,2]]]


        y_real = (real_tau - self.freqs.DPi1*(self.freqs.shift))
        X_real = np.c_[y_real, self.inp_freqs]

        # now compute distance between original frequencies with the frequencies under a new model hypothesis

        dist = 0
        
        dist += kdtree_match(X_model=X, X_real=X_real,
                                       DPi1=self.freqs.DPi1, numax=self.freqs.numax,
                                       delta_nu=self.freqs.delta_nu, 
                                       weights=self.weights)

        
        return dist
    
class freq_tau_rot_doublet:
    def __init__(self, init_dpi, init_q, freqs_class, inp_freqs, weights):
        self.lb = np.array([init_dpi*0.98, max(0, init_q-0.1), -0.4, 0.1]) #or another judicious choice
        self.ub = np.array([init_dpi*1.02, init_q+0.1, 0.4, 1.0]) ## RGB
        
#         self.lb = np.array([init_dpi*0.98, 0.05, -0.5, 0.0]) 
#         self.ub = np.array([init_dpi*1.02, 0.6, 0.2, 0.1]) # HeB
        self.freqs = freqs_class
        self.inp_freqs = inp_freqs
        self.weights = weights
        
    def __call__(self, x):
        assert len(x) == 4
        assert x.ndim == 1
        assert np.all(x <= self.ub) and np.all(x >= self.lb)
        
        dpi1, coupling, eps_g, split = x[0], x[1], x[2], x[3]
        
        params = {'calc_l0': True, # copy of params with DPi1 set to candidate dpi from loop
            'calc_l2': True, 
            'calc_l3': False, 
            'calc_nom_l1': True, 
            'calc_mixed': True, 
            'calc_rot': False,
            'DPi1': dpi1,
            'coupling': coupling, 
            'eps_g': eps_g, 
            'split_core': split,
            'split_env': 0.0,
            'l': 1,
            }
    
        self.freqs(params)

        self.freqs.generate_tau_values()

        # Compute tau from the zeta value just computed
        real_tau = mixed_modes_utils.peaks_stretched_period(self.inp_freqs, 
                                                                 self.freqs.frequency, 
                                                                 self.freqs.tau)

        freqs_p1 = self.freqs.l1_mixed_freqs + self.freqs.l1_zeta * split
        freqs_n1 = self.freqs.l1_mixed_freqs - self.freqs.l1_zeta * split

        tau_p1 = mixed_modes_utils.peaks_stretched_period(freqs_p1, self.freqs.frequency, self.freqs.tau)
        tau_n1 = mixed_modes_utils.peaks_stretched_period(freqs_n1, self.freqs.frequency, self.freqs.tau)

        model_freqs = np.c_[self.freqs.l1_mixed_freqs, freqs_p1, freqs_n1]
        model_tau = np.c_[self.freqs.l1_mixed_tau, tau_p1, tau_n1]   

#         # #doublet case
        X = np.c_[np.r_[model_tau[:,1] - self.freqs.shift * self.freqs.DPi1, 
                        model_tau[:,2] - self.freqs.shift * self.freqs.DPi1], 
                  np.r_[model_freqs[:,1], 
                        model_freqs[:,2]]]


        y_real = (real_tau - self.freqs.DPi1*(self.freqs.shift))
        X_real = np.c_[y_real, self.inp_freqs]

        # now compute distance between original frequencies with the frequencies under a new model hypothesis

        dist = 0
        
        dist += kdtree_match(X_model=X, X_real=X_real,
                                       DPi1=self.freqs.DPi1, numax=self.freqs.numax,
                                       delta_nu=self.freqs.delta_nu, 
                                       weights=self.weights)

        
        return dist
    
class freq_tau_rot_singlet:
    def __init__(self, init_dpi, init_q, freqs_class, inp_freqs, weights):
        self.lb = np.array([init_dpi*0.95, max(0, init_q-0.1), -0.5])
        self.ub = np.array([init_dpi*1.05, init_q+0.1, 0.5])
        self.freqs = freqs_class
        self.inp_freqs = inp_freqs
        self.weights = weights

    def __call__(self, x):
        assert len(x) == 3
        assert x.ndim == 1
        assert np.all(x <= self.ub) and np.all(x >= self.lb)
        
        dpi1, coupling, eps_g = x[0], x[1], x[2]
        
        
        params = {'calc_l0': True, # copy of params with DPi1 set to candidate dpi from loop
            'calc_l2': True, 
            'calc_l3': False, 
            'calc_nom_l1': True, 
            'calc_mixed': True, 
            'calc_rot': False,
            'DPi1': dpi1,
            'coupling': coupling, 
            'eps_g': eps_g, 
            'split_core': 0.0,
            'split_env': 0.0,
            'l': 1,
            }
    
        self.freqs(params)

        self.freqs.generate_tau_values()

        # Compute tau from the zeta value just computed
        real_tau = mixed_modes_utils.peaks_stretched_period(self.inp_freqs, 
                                                                 self.freqs.frequency, 
                                                                 self.freqs.tau)


        model_freqs = self.freqs.l1_mixed_freqs.reshape(-1,1)
        model_tau = self.freqs.l1_mixed_tau.reshape(-1,1)

        # singlet case  
        X = np.c_[model_tau[:,0], model_freqs[:,0]]    
        y_real = (real_tau - self.freqs.DPi1*(self.freqs.shift))
        X_real = np.c_[y_real, self.inp_freqs]


        y_real = (real_tau - self.freqs.DPi1*(self.freqs.shift))
        X_real = np.c_[y_real, self.inp_freqs]

        # now compute distance between original frequencies with the frequencies under a new model hypothesis

        dist = 0


        dist += kdtree_match(X_model=X, X_real=X_real,
                                       DPi1=self.freqs.DPi1, numax=self.freqs.numax,
                                       delta_nu=self.freqs.delta_nu, 
                                       weights=self.weights)

        return dist

## Optimizer and plotting function

In [ ]:
def optimize_rotation(init_dpi, init_q, freqs_class, inp_freqs, loss_list_index, weights, verbose=True):
    num_components = loss_list_index + 1
    print('Number of Components: ', num_components)
    
    if num_components == 1:
        z = freq_tau_rot_singlet(init_dpi, init_q, freqs_class, inp_freqs, weights)
    elif num_components == 2:
        z = freq_tau_rot_doublet(init_dpi, init_q, freqs_class, inp_freqs, weights)
    elif num_components == 3:
        z = freq_tau_rot_triplet(init_dpi, init_q, freqs_class, inp_freqs, weights)
    else:
        raise ValueError
        
    turbo1_rot = TurboM(
    f=z,  # Handle to objective function
    lb=z.lb,  # Numpy array specifying lower bounds
    ub=z.ub,  # Numpy array specifying upper bounds
    n_init=50,  # Number of initial bounds from an Symmetric Latin hypercube design
    max_evals=5000,  # Maximum number of evaluations
    n_trust_regions=10,  # Number of trust regions
    batch_size=100,  # How large batch size TuRBO uses
    verbose=verbose,  # Print information from each batch
    use_ard=True,  # Set to true if you want to use ARD for the GP kernel
    max_cholesky_size=2000,  # When we switch from Cholesky to Lanczos
    n_training_steps=50,  # Number of steps of ADAM to learn the hypers
    min_cuda=64,  # Run on the CPU for small datasets
    device="cuda",  # "cpu" or "cuda"
    dtype="float64",  # float64 or float32
    )


#     turbo1_rot.failtol = 20 # double the tolerance
    turbo1_rot.n_cand = 5000
    turbo1_rot.optimize()
    X_rot = turbo1_rot.X  # Evaluated points
    fX_rot = turbo1_rot.fX  # Observed values
    ind_best_rot = np.argmin(fX_rot)
    f_best_rot, x_best_rot = fX_rot[ind_best_rot], X_rot[ind_best_rot, :]
    return x_best_rot, f_best_rot, X_rot, fX_rot.ravel()


def plot_solution(freqs_class, inp_freqs, prepped_l1_peaks, cand_dpi, cand_q, cand_eps_g, cand_rot, rot_index, axa, axb,
                 label='BOpt', idx=0):
    # Set up frequencies class
    freqs_dummy = deepcopy(freqs_class)
    splitting=cand_rot
    
    params = {'calc_l0': True, 
                'calc_l2': True, 
                'calc_l3': False, 
                'calc_nom_l1': True, 
                'calc_mixed': True, 
                'calc_rot': False, 
                'DPi1': cand_dpi,
                'coupling': cand_q,
                'eps_g': cand_eps_g,
                'l': 1, 
                }

    freqs_dummy(params)
    freqs_dummy.generate_tau_values()
    cand_dpi = freqs_dummy.DPi1

    real_tau = mixed_modes_utils.peaks_stretched_period(inp_freqs, 
                                                             freqs_dummy.frequency, 
                                                             freqs_dummy.tau)
    real_tau = real_tau - freqs_dummy.DPi1*(freqs_dummy.shift)


    freqs_p1 = freqs_dummy.l1_mixed_freqs + freqs_dummy.l1_zeta * splitting
    freqs_n1 = freqs_dummy.l1_mixed_freqs - freqs_dummy.l1_zeta * splitting

    tau_p1 = mixed_modes_utils.peaks_stretched_period(freqs_p1, freqs_dummy.frequency, freqs_dummy.tau)
    tau_n1 = mixed_modes_utils.peaks_stretched_period(freqs_n1, freqs_dummy.frequency, freqs_dummy.tau)

    model_freqs = np.c_[freqs_dummy.l1_mixed_freqs, freqs_p1, freqs_n1]
    model_tau = np.c_[freqs_dummy.l1_mixed_tau, tau_p1, tau_n1]

    plot_tau = np.mod(real_tau, freqs_dummy.DPi1)
    plot_tau[plot_tau > freqs_dummy.DPi1/2] -= freqs_dummy.DPi1
    
    if rot_index == 0:
        X = np.c_[model_tau[:,0], model_freqs[:,0]]
    elif rot_index == 1:
        X = np.c_[np.r_[model_tau[:,1] - freqs_dummy.shift * freqs_dummy.DPi1, 
                        model_tau[:,2] - freqs_dummy.shift * freqs_dummy.DPi1], 
                  np.r_[model_freqs[:,1], 
                        model_freqs[:,2]]] 
    elif rot_index == 2:  
        X = np.c_[np.r_[model_tau[:,0],
                        model_tau[:,1] - freqs_dummy.shift * freqs_dummy.DPi1, 
                        model_tau[:,2] - freqs_dummy.shift * freqs_dummy.DPi1], 
                  np.r_[model_freqs[:,0],
                        model_freqs[:,1], 
                        model_freqs[:,2]]]      
    else:
        raise ValueError
        

    plot_model_tau = np.mod(X[:,0], freqs_dummy.DPi1)
    plot_model_tau[plot_model_tau > freqs_dummy.DPi1/2] -=  freqs_dummy.DPi1

    axa.scatter(plot_tau, inp_freqs, s=15,
            c='royalblue', edgecolor='k', alpha=0.8,
               label='$\\Delta\\Pi_1$: %.2f, $q$: %.2f, $\\epsilon_g$: %.2f, $\\delta_{\\mathrm{rot}}$: %.2f'
               %(cand_dpi, cand_q, cand_eps_g, cand_rot))
    axa.set_ylim(np.min(inp_freqs)-2, np.max(inp_freqs)+2)
    axa.set_ylabel('$\\nu$ ($\\mu$Hz)', fontsize=16)
    axa.set_xlabel('$\\tau$ mod $\\Delta\\Pi_1$', fontsize=16)
    axa.set_xlim(-cand_dpi/2, cand_dpi/2)
    axa.set_title(label, fontsize=15)
    axa.legend(prop={'size': 15})
    axa.tick_params(labelsize=15)


    axb.scatter(plot_model_tau, X[:,1], s=10, c='r', alpha=0.25, label='Model')
    axb.scatter(plot_tau, inp_freqs, s=np.power(prepped_l1_peaks['height'].values, 1.5),
             c='royalblue', edgecolor='k', alpha=0.8, label='Observed')
    axb.set_ylim(np.min(inp_freqs)-2, np.max(inp_freqs)+2)
    axb.set_ylabel('$\\nu$ ($\\mu$Hz)', fontsize=16)
    axb.set_xlabel('$\\tau$ mod $\\Delta\\Pi_1$', fontsize=16)
    axb.set_xlim(-cand_dpi/2, cand_dpi/2)
    axa.tick_params(labelsize=15)
    
    ### Calculating the Hungarian loss difference upon removal of specific rotational components ###
    
    X_no0 = np.c_[np.r_[model_tau[:,1] - freqs_dummy.shift * freqs_dummy.DPi1, 
                    model_tau[:,2] - freqs_dummy.shift * freqs_dummy.DPi1], 
              np.r_[model_freqs[:,1], 
                    model_freqs[:,2]]]
    X_nop1 = np.c_[np.r_[model_tau[:,0],
                    model_tau[:,2] - freqs_dummy.shift * freqs_dummy.DPi1], 
              np.r_[model_freqs[:,0],
                    model_freqs[:,2]]]
    X_non1 = np.c_[np.r_[model_tau[:,0],
                    model_tau[:,1] - freqs_dummy.shift * freqs_dummy.DPi1], 
              np.r_[model_freqs[:,0],
                    model_freqs[:,1]]]

    X_list = [X, X_no0, X_nop1, X_non1]
    condition = ['m=0 Removal', 'm=+1 Removal', 'm=-1 Removal']
    pct_diff_vec = []
    for i, inp in enumerate(X_list):
        c1 = np.vstack((inp[:,0]/freqs_dummy.DPi1, (inp[:,1]-freqs_dummy.numax)/freqs_dummy.delta_nu)).T#(X_t[:,1]-freqs.numax)/freqs.delta_nu )).T
        c2 = np.vstack((real_tau/freqs_dummy.DPi1, (inp_freqs-freqs_dummy.numax)/freqs_dummy.delta_nu)).T


        tree = KDTree(c1)
        nearest_dists, nearest_idx = tree.query(c2, k=1)
        kost = np.median(nearest_dists)
        if i == 0:
            prime_cost = kost
        else:
            pct_diff = np.abs((kost-prime_cost)*100./prime_cost)
            pct_diff_vec.append(np.round(pct_diff, 2))
    
    axb.set_title('KIC: %d, [$\\delta_0$, $\\delta_{+1}$, $\\delta_{-1}$]: %s' %(idx, pct_diff_vec), fontsize=15)
    return pct_diff_vec

## Main looping function

In [ ]:
def bopt_rotation(data):
    name, row = data
    iz, init_dpi, init_q = row['KIC'], row['dpi'], row['q']
    
    header_df = pd.read_csv(abba_folder + '%d.modes.dat' %iz, 
                            delim_whitespace=True, header=0, skiprows = 4, nrows=1)
    nmx = header_df['fmax'].values[0]
    dnu = header_df['dnu'].values[0]
    evo = header_df['evo'].values[0]
    alpha = header_df['alpha'].values[0]
    f0 = header_df['f0_c'].values[0]
    eps = np.mod(f0, dnu)/dnu
    
    print('KIC %d, Init DPi1 %.2f, Init q: %.2f' %(iz, init_dpi, init_q))
    if iz in out_file.iloc[:,0].values:
        return None
        
    if eps < 0.5:
        eps += 1
        
    peaks = pd.read_csv(abba_folder + '%d.modes.dat' %iz, 
                            delim_whitespace=True, header=0, skiprows = 7)
    peaks = peaks.rename(columns={'amp': 'height'})
    
    peaks['linewidth'] = 1/(1*np.pi*peaks['tau']*0.0864)
    peaks['weights'] = (peaks['height']/np.sum(peaks['height']))*1000.
    
    evidence_cut=0.05
    
    peaks['x'] = ((peaks['freq'] % dnu - eps) / dnu) % 1
    # Don't want to include any modes near l=0 or 2s, this is why this and the step in the next cell is performed.
    x_range = [(np.minimum(np.min(peaks.loc[peaks['l'] == 0, 'x']), np.min(peaks.loc[peaks['l'] == 2, 'x'])) - 0.05) % 1,
               (np.maximum(np.max(peaks.loc[peaks['l'] == 0, 'x']), np.max(peaks.loc[peaks['l'] == 2, 'x'])) + 0.05) % 1]
    
    l1_peaks = peaks.loc[(peaks.l == 1) | ~np.isfinite(peaks.l), ]
    l1_peaks['x'] = ((l1_peaks['freq'] % dnu - eps) / dnu) % 1
    
    if x_range[0] < x_range[1]:
        l1_peaks = l1_peaks.loc[(l1_peaks['x'] < x_range[1]) | (l1_peaks['x'] > x_range[0]), ] # changed to OR for HeB
    else:
        l1_peaks = l1_peaks.loc[(l1_peaks['x'] > x_range[1]) & (l1_peaks['x'] < x_range[0]), ] # this is AND for RGB
    prepped_l1_peaks = l1_peaks.loc[(l1_peaks['ev1'] > evidence_cut), ]
    
    try:
        pds = pd.read_csv(abba_psd_folder + 'kplr%d.ts.fft.pds.bgcorr' %iz,
                              header=None, delim_whitespace=True)
    except:
        return None
    pds.columns = ['frequency', 'power']
    # Only keep pds around oscillations
    sigmaEnv = (0.59 * (nmx ** 0.9))/2
    pds = pds.loc[abs(pds['frequency'].values - nmx) < 3 * sigmaEnv, ]
    peaks = peaks.loc[abs(peaks.freq.values - nmx) < 3*sigmaEnv, ]
    
    l023_peaks = peaks.loc[(peaks.l == 0) | (peaks.l == 2) | (peaks.l == 3), ]
    l0_peaks = peaks.loc[(peaks.l==0), ]
    l1_peaks = peaks.loc[(peaks.l == 1) | (np.isfinite(peaks.l) == False)]
    l2_peaks = peaks.loc[(peaks.l==2), ]
    
    pds_l023_removed = pds.assign(power = pds.power / fit_model(pds, l023_peaks))
    
    try:
        freqs_class = frequencies.Frequencies(frequency=pds_l023_removed.frequency.values,
                                    numax=nmx, 
                                    delta_nu=dnu, 
                                    epsilon_p=eps,
                                    alpha=alpha)
    except:
        return None
    
    inp_freqs = prepped_l1_peaks['freq'].values
    if len(inp_freqs) == 0:
        pass
    else:
        loss_list_index = 2 # automatically assume triplet

        init_time = timer.time()
        best_sol, best_loss, sampled_points, losses = optimize_rotation(init_dpi, init_q, freqs_class, inp_freqs, loss_list_index,
                                           weights=prepped_l1_peaks['weights'].values, verbose=False)
        print('Time Taken to Optimize Rotation: ', timer.time()-init_time)
        if loss_list_index == 0:
            rotx = -99
        else:
            rotx = np.around(best_sol, 3)[3]

        fig = plt.figure(figsize=(12, 7))
        ax1, ax2 = fig.add_subplot(121), fig.add_subplot(122)
        pct_diff_vec = plot_solution(freqs_class, inp_freqs, prepped_l1_peaks,
                                     cand_dpi=np.around(best_sol, 3)[0],
                      cand_q=np.around(best_sol, 3)[1],
                      cand_eps_g=np.around(best_sol, 3)[2],
                      cand_rot=rotx, 
                      rot_index=loss_list_index, 
                      axa=ax1, axb=ax2,
                     label='BOpt Solution', idx=iz)

        plt.tight_layout(h_pad=0)
        plt.savefig(image_folder + '%d.png' %iz)
        plt.clf()
        plt.close()
        np.savez_compressed(samples_folder + '%d.npz' %iz,
                           kic=iz, samples=sampled_points, loss=losses)

        with open(output_file, 'a') as writer:
            writer.write(str(int(iz))+','+str(np.around(best_sol, 3)[0])+','+str(np.around(best_sol, 3)[1])+','\
                         +str(np.around(best_sol, 3)[2])+','+str(np.around(rotx, 3))+','+ str(np.around(best_loss, 3)[0])+','\
                         +str(np.round(pct_diff_vec[0], 2))+','+str(np.round(pct_diff_vec[1], 2))+','+str(np.round(pct_diff_vec[2], 2)) +'\n')


## Running the loop in parallel

In [ ]:
results = Parallel(n_jobs=4)(delayed(bopt_rotation)(i) for i in tqdm(summary_df.iterrows(), total=len(summary_df)))